名称：持续时间长度
类型：时间、度量
单位：小时

In [260]:
import pandas as pd
import utils
import numpy as np

def onesensorFeatures(df_act):
    pass

def sensorpairFeatures(df_act):
    pass
    

def sensorTimeFeatures(df_act):
    pass


def timeFeatures(df_act):
    """
    返回时间相关特征
    时间间隔（delta，分钟）、是否跨天（is_two_day）,是否是工作日？周六？周日？开始时间,结束时间（小时）
    """
    from datetime import datetime,timedelta
    ts = np.array(df_act.time)
    ds = np.array(df_act.date)
    
    sy,sm,sd = ds[0].split("-")
    sh,sminutes = ts[0].split(":")[0:-1]
    ss,sns = ts[0].split(":")[-1].split(".")
    
    ey,em,ed = ds[-1].split("-")
    eh,eminutes = ts[-1].split(":")[0:-1]
    es,ens = ts[-1].split(":")[-1].split(".")
    
    starttime = datetime(int(sy),int(sm),int(sd),int(sh),int(sminutes),int(ss),int(sns))
    endtime = datetime(int(ey),int(em),int(ed),int(eh),int(eminutes),int(es),int(ens))
    
    
    delta = (endtime - starttime).total_seconds()
    delta = int(delta)


#     是否跨天
    is_two_day = 0
    if ds[0] != ds[-1]:
        is_two_day = 1

    

    
#     是否是周六？周日？工作日？
    day_of_week = pd.DatetimeIndex(df_act.date).dayofweek[-1] + 1
    is_workingday = 1
    is_sat = 0
    is_sun = 0
    if day_of_week == 6:
        is_workingday = 0
        is_sat = 1
    if day_of_week == 7:
        is_workingday = 0
        is_sun = 1
    
#     开始时间
    start_hour = int(sh)
#     结束时间
    end_hour = int(eh)

    return np.array([delta,is_two_day,is_workingday,is_sat,is_sun,start_hour,end_hour])

    

def dataframeActivity(activity):
    """
    输入：一个完整的观测序列
    输出：dataframe类型的数据结构
    
    """
    df_act = pd.DataFrame(activity[1:],columns=['date','time','sensor','value'])
    
    return df_act
    
    

    

In [263]:
filename = "./data/ann.txt"
activities = utils.ExtractActivities(filename)

tfs = []
sfs = []
spfs = []
stfs = []

for act in activities:
    df = dataframeActivity(act)
    tf = timeFeatures(df)
    tfs.append(tf)
df_time_features = pd.DataFrame(tfs,columns=["length",'istwoday',"isworkingday","issat","issun","starthour","endhour"])






[ ['Sleep', ['2011-06-15', '02:13:20.812509', 'M007', 'ON'], ['2011-06-15', '02:13:21.919635', 'M007', 'OFF'], ['2011-06-15', '02:35:58.552127', 'T102', '25'], ['2011-06-15', '02:39:08.012618', 'BATV006', '9500'], ['2011-06-15', '02:59:44.032083', 'BATV019', '9500'], ['2011-06-15', '03:12:44.112187', 'BATV105', '3100'], ['2011-06-15', '03:20:30.966622', 'BATV008', '9460'], ['2011-06-15', '03:30:27.009192', 'BATV009', '9440'], ['2011-06-15', '03:49:34.216414', 'BATV011', '9480'], ['2011-06-15', '03:57:08.06642', 'BATV106', '3120'], ['2011-06-15', '04:07:54.361645', 'BATV002', '9520'], ['2011-06-15', '04:16:09.529547', 'BATV020', '9480'], ['2011-06-15', '04:25:16.566478', 'BATV104', '3170'], ['2011-06-15', '04:40:18.714468', 'BATV005', '9480'], ['2011-06-15', '04:50:25.941993', 'BATV004', '9360'], ['2011-06-15', '04:54:49.92984', 'T102', '24'], ['2011-06-15', '05:14:40.081278', 'T102', '23'], ['2011-06-15', '05:37:53.817078', 'LS002', '3'], ['2011-06-15', '05:38:52.222794', 'LS022', '1']

In [250]:
print activities[4]

['Morning_Meds', ['2011-06-15', '06:56:26.7509', 'MA023', 'ON'], ['2011-06-15', '06:56:26.861011', 'M014', 'ON'], ['2011-06-15', '06:56:27.794793', 'BATV023', '9360'], ['2011-06-15', '06:56:27.845202', 'MA023', 'OFF'], ['2011-06-15', '06:56:28.550615', 'MA023', 'ON'], ['2011-06-15', '06:56:29.638481', 'M008', 'ON']]
